# Imports

In [1]:
import numpy as np
from prettytable import PrettyTable
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import xraydb

import sys
script_dir = '/Users/andrew/local_data_analysis/repos/xraytools/'
sys.path.append(script_dir)
import xraytools
#iPython magic to make interactive plots work
%matplotlib widget

# Sample Detector Distance Calculators

Given a maximum q-value how far away can I put my detector?

In [2]:
### User Inputs ###
length_y = 180 # (mm) total detector height (including detector gaps)
length_x = 170 # (mm) total detector width (including detector gaps)
energy = 16100 # (eV) Energy of X-rays
max_q = 0.8 # maximum q you would like to reach

### Calculations ###
xraytools.calc_sdd(length_y, length_x, energy, max_q)

Table showing SDDs (in mm) to put Q=0.80 Å-1 at the edge/corner of the detector
+----------------+-------------+-------------+-----------+-------------+
|                | Center Beam | Bottom Beam | Side Beam | Corner Beam |
+----------------+-------------+-------------+-----------+-------------+
| SDD (max ring) |     864     |     864     |    914    |     1727    |
|  SDD (corner)  |     1258    |     2023    |    1954   |     2516    |
+----------------+-------------+-------------+-----------+-------------+


Given an SDD and beam position what is the max q I will be able to detect?

In [ ]:
### User Inputs ###
length_y = 180 # (mm) total detector height (including detector gaps)
length_x = 170 # (mm) total detector width (including detector gaps)
energy = 16100 # (eV) Energy of X-rays
sdd = 8200 # (mm) sample detector distance
bs_radius = 1 # (mm) radius of beamstop or minimum radius from beam center to acheive good data

xraytools.calc_qrange(length_y, length_x, bs_radius, energy, sdd)

# Critical Angle Calculators

Calculate a single critical angle

In [ ]:
stoichiometry = 'Si'
density = 2.33 #g/cc
energy = 8000 #eV
crit_angle = xraytools.calc_critical_angle(energy, stoichiometry, density)

Calculate a critical angles for many energies and materials

In [ ]:
energies = np.linspace(8000,10000,5) #energies of interest in eV
stoichiometries = ['C2H4','Si']
densities = [1, 2.33] #g/cc
crit_angles = xraytools.calc_critical_angle_table(energies, stoichiometries, densities)

# XEFI Calculator

In [ ]:
# Example cell calculating and plotting XEFI for a PM6 film on Si:

plt.close('all')
energy = 13500  # eV of GIWAXS data
film_stoichiometry = 'C68H76F2O2S8'  # PM6 stoichiometry
sub_stoichiometry = 'Si'  # on silicon substrate
film_density = 1  # ~ density of PM6
sub_density = 2.33  # density of Si
sampthick = 180  # set film thickness
aoi = 0.095  # center of aoi range
aoi_extent = 0.03  # extent ± of aoi range

aois, depth, xefi = xraytools.calc_xefi(energy, 
                                    film_stoichiometry, 
                                    film_density, 
                                    sub_stoichiometry, 
                                    sub_density,
                                    sampthick, 
                                    aoi,
                                    aoi_extent)

xefi_mag = np.abs(xefi)  # Electric field is complex, get magnitude


# Quick plot check 
cmin = np.quantile(EE_mag, 0.05)
cmax = np.quantile(EE_mag, 1)

fig, ax = plt.subplots(figsize=(5,3), dpi=150, tight_layout=True)

im = ax.imshow(
    xefi_mag, 
    origin='upper', 
    extent=[aois[0],aois[-1],depth[-1],depth[0]], 
    aspect='auto',
    norm=plt.Normalize(cmin,cmax),
    cmap=plt.cm.terrain
)
ax.set(xlabel='Incident angle [°]', ylabel= 'Film depth [nm]')
# ax.yaxis.set_minor_locator(MultipleLocator(10))  # from matplotlib.ticker import MultipleLocator
# ax.yaxis.set_major_locator(MultipleLocator(20))  # to set specific tick intervals

# Add colorbar
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('XEFI', rotation=270, labelpad=10)

plt.show()

# Yoneda peak q-position calculator

In [ ]:
stoichiometry = 'C2H4'
density = 1 #g/cc
energy = 13500 #eV
incident_deg = 0.15 #deg
q_val = xraytools.calc_yoneda_material(incident_deg, energy, stoichiometry, density)

# X-ray edges lookup
Get all X-ray edges for a given element

In [ ]:
edges = xraytools.find_xray_edges('Pb')

# Material Attenuation and Transmission Calculators
These calculators will take your given material stack of stoichiometries and densities to calculate attenuation length or transmission

### Calculate Material(s) Attenuation Lengths

In [ ]:
energies = np.linspace(12000,24000,13) #energies of interest in eV
stoichiometries = ['C2H4','Si']
densities = [1, 2.33] #g/cc
xraytools.calc_mu_grid(energies, stoichiometries, densities, table=True, plot=True)

### Calculate Material(s) X-ray Transmission

In [ ]:
energies = np.linspace(12000,24000,13) #energies of interest in eV
stoichiometries = ['C2H4','Si']
densities = [1, 2.33] #g/cc
thicknesses = [.373, .101] #thicknesses in mm
xraytools.calc_transmission_grid(energies, stoichiometries, densities, thicknesses, verbose=True, table=True, plot=True)

# Solution Transmission Calculator
This calculator will take your given solvent and solute chemistry along with the mass/volume of solute added to calculate transmission

*Note: this calculator assumes you were lazy when you made your solution and reported concentration as milligrams added over ml of solvent. If you made the solution rigorously then this calculator will be slightly off*

In [ ]:
energies = np.linspace(12000,24000,10) 
solvent_stoich =  'C2H6OS'
solvent_dens =  1.1
solute_stoich =  'PbI2'
solute_dens =  6.16
mg_per_ml =  100 #mg/ml
cap_diam = 1 #mm
xraytools.calc_solution_transmission(energies, solvent_stoich, solvent_dens, solute_stoich, solute_dens, mg_per_ml, cap_diam, table=True, plot=False)